In [1]:
# https://hironsan.hatenablog.com/entry/named-entity-recognition-with-elmo
# を実装してみる

In [1]:
import argparse
import os

import numpy as np

from allennlp.modules.elmo import Elmo, batch_to_ids

In [3]:
options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"
elmo = Elmo(options_file, weight_file, 2, dropout=0)


  3%|▎         | 10845184/374434792 [00:16<06:45, 896135.65B/s]


  8%|▊         | 28687360/374434792 [00:31<03:35, 1602326.53B/s]


 12%|█▏        | 44494848/374434792 [00:47<07:18, 752936.49B/s]


 16%|█▌        | 59813888/374434792 [01:03<04:19, 1212280.96B/s]


 20%|██        | 74910720/374434792 [01:18<03:37, 1375750.20B/s]


...

 84%|████████▍ | 314806272/374434792 [05:09<01:11, 833894.86B/s]


 88%|████████▊ | 328976384/374434792 [05:25<00:54, 837945.48B/s]


 92%|█████████▏| 344591360/374434792 [05:40<00:19, 1519823.00B/s]


 98%|█████████▊| 365742080/374434792 [05:55<00:11, 784526.19B/s]


100%|██████████| 374434792/374434792 [06:04<00:00, 1028277.99B/s]

In [4]:
sentences = [['First', 'sentence', '.'], ['Another', '.']]
character_ids = batch_to_ids(sentences)
embeddings = elmo(character_ids)

In [5]:
embeddings["elmo_representations"][0]

tensor([[[ 0.1474, -0.1475,  0.1376,  ...,  0.0270, -0.4051, -0.0498],
         [ 0.2394,  0.0769,  0.4126,  ..., -0.1671, -0.1707,  0.3884],
         [-0.7602, -0.4944, -0.5355,  ..., -0.0803,  0.0361,  0.1128]],

        [[ 0.2603, -0.4437,  0.2726,  ..., -0.0830, -0.1522, -0.1361],
         [-0.7772, -0.4294, -0.2651,  ..., -0.0803,  0.0361,  0.1128],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       grad_fn=<DropoutBackward>)

In [6]:
embeddings["elmo_representations"][0].shape

torch.Size([2, 3, 1024])

In [ ]:
# ここまでelmoが何かの確認

In [62]:
def load_data_and_labels(filename):
    sents, labels = [], []
    words, tags = [], []
    with open(filename) as f:
        for line in f:
            line = line.rstrip()
            if line:
                word, tag = line.split('\t')
                words.append(word)
                tags.append(tag)
            else:
                sents.append(words)
                labels.append(tags)
                words, tags = [], []
                
    return sents, labels

In [63]:
train_file = 'data/conll2003/en/ner/train.txt'
valid_file = 'data/conll2003/en/ner/valid.txt'

x_train, y_train = load_data_and_labels(train_file)
x_valid, y_valid = load_data_and_labels(valid_file)

In [64]:
x_train[0]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [65]:
UNK = '<UNK>'
PAD = '<PAD>'

vocab_word = {PAD: 0, UNK: 1}
vocab_char = {PAD: 0, UNK: 1}
vocab_label = {PAD: 0}

In [66]:
for sent in x_train:
    for w in sent:
        # create char dictionary.
        for c in w:
            if c in vocab_char:
                continue
            vocab_char[c] = len(vocab_char)

        # create word dictionary.
        if w in vocab_word:
            continue
        vocab_word[w] = len(vocab_word)

# create label dictionary.
for labels in y_train:
    for tag in labels:
        if tag in vocab_label:
            continue
        vocab_label[tag] = len(vocab_label)

In [67]:
def get_char_sequences(x):
    chars = []
    for sent in x:
        chars.append([list(w) for w in sent])

    return chars

In [68]:
x_train_chars = get_char_sequences(x_train)
x_valid_chars = get_char_sequences(x_valid)

In [69]:
x_train_chars[0]

[['E', 'U'],
 ['r', 'e', 'j', 'e', 'c', 't', 's'],
 ['G', 'e', 'r', 'm', 'a', 'n'],
 ['c', 'a', 'l', 'l'],
 ['t', 'o'],
 ['b', 'o', 'y', 'c', 'o', 't', 't'],
 ['B', 'r', 'i', 't', 'i', 's', 'h'],
 ['l', 'a', 'm', 'b'],
 ['.']]

In [70]:
def transform_word(x):
    seq = []
    for sent in x:
        word_ids = [vocab_word.get(w, vocab_word[UNK]) for w in sent]
        seq.append(word_ids)

    return seq


In [71]:
x_train_words = transform_word(x_train)
x_valid_words = transform_word(x_valid)

In [72]:
x_train_words[0]

[2, 3, 4, 5, 6, 7, 8, 9, 10]

In [73]:
def transform_char(x):
    seq = []
    for sent in x:
        char_seq = []
        for w in sent:
            char_ids = [vocab_char.get(c, vocab_char[UNK]) for c in w]
            char_seq.append(char_ids)
        seq.append(char_seq)
    
    return seq

In [74]:
x_train_chars = transform_char(x_train)
x_valid_chars = transform_char(x_valid)

In [75]:
x_train_chars[0]

[[2, 3],
 [4, 5, 6, 5, 7, 8, 9],
 [10, 5, 4, 11, 12, 13],
 [7, 12, 14, 14],
 [8, 15],
 [16, 15, 17, 7, 15, 8, 8],
 [18, 4, 19, 8, 19, 9, 20],
 [14, 12, 11, 16],
 [21]]

In [76]:
def transform_label(y):
    seq = []
    for labels in y:
        tag_ids = [vocab_label[tag] for tag in labels]
        seq.append(tag_ids)
        
    return seq

In [77]:
y_train = transform_label(y_train)

In [78]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical


def pad_char(sequences):
    maxlen_word = max(len(max(seq, key=len)) for seq in sequences)
    maxlen_seq = len(max(sequences, key=len))
    sequences = [list(seq) + [[] for i in range(max(maxlen_seq - len(seq), 0))] for seq in sequences]

    return np.array([pad_sequences(seq, padding='post', maxlen=maxlen_word) for seq in sequences])

x_train_words = pad_sequences(x_train_words, padding='post')
x_valid_words = pad_sequences(x_valid_words, padding='post')
x_train_chars = pad_char(x_train_chars)
x_valid_chars = pad_char(x_valid_chars)
y_train = pad_sequences(y_train, padding='post')
y_train = to_categorical(y_train, len(vocab_label))

In [79]:
char_vocab_size = len(vocab_char)
char_emb_size = 50
char_lstm_units = 25
word_vocab_size = len(vocab_word)
word_emb_size = 100
word_lstm_units = 100
num_tags = len(vocab_label)

In [80]:
from keras.layers import Input

word_ids = Input(shape=(None,), dtype='int32')
char_ids = Input(shape=(None, None), dtype='int32')
elmo_embeddings = Input(shape=(None, 1024), dtype='float32')

In [81]:
import keras.backend as K
from keras.layers import LSTM, Embedding, Bidirectional, TimeDistributed
from keras.layers.merge import Concatenate

char_embeddings = Embedding(input_dim=char_vocab_size,
                            output_dim=char_emb_size,
                            mask_zero=True
                            )(char_ids)
char_embeddings = TimeDistributed(Bidirectional(LSTM(char_lstm_units)))(char_embeddings)

In [82]:
word_embeddings = Embedding(input_dim=word_vocab_size,
                            output_dim=word_emb_size,
                            mask_zero=True)(word_ids)

In [83]:
x = Concatenate(axis=-1)([word_embeddings, char_embeddings, elmo_embeddings])
x = Bidirectional(LSTM(units=word_lstm_units, return_sequences=True))(x)

In [89]:
from keras.layers import Dense
from layers import CRF

x = Dense(word_lstm_units, activation='tanh')(x)
x = Dense(num_tags)(x)
crf = CRF(10)
pred = crf(x)

In [90]:
from keras.models import Model
model = Model(inputs=[word_ids, char_ids, elmo_embeddings], outputs=[pred])
model.compile(loss=crf.loss_function, optimizer='adam')

In [91]:
model.fit([x_train_words, x_train_chars], y_train)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 2 arrays: [array([[    2,     3,     4, ...,     0,     0,     0],
       [   11,    12,     0, ...,     0,     0,     0],
       [   13,    14,     0, ...,     0,     0,     0],
       ...,
       [ 4126,   69...